<a href="https://colab.research.google.com/github/PIYALI-bhunia/DEEP_LEARNING/blob/main/classifier_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset=pd.read_csv("/content/drive/MyDrive/ML DATASET/diabetes.csv")
dataset.shape

(768, 9)

In [3]:
X=dataset.iloc[:, :-1]
y=dataset.iloc[:, -1]

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
X_train.shape

(614, 8)

In [5]:
!pip install keras-tuner

     |████████████████████████████████| 98 kB 4.7 MB/s 


In [17]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LeakyReLU,PReLU,ELU
from keras.layers import *
import keras_tuner as kt
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

In [7]:
def build_model(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(8,)))
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='binary_crossentropy',
        metrics=['accuracy'])
    return model

In [8]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='project',
    project_name='diabetes')


In [9]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
from sklearn.preprocessing import MinMaxScaler


In [11]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [12]:
tuner.search(X_train_scaled, y_train,
             epochs=100,
             validation_data=(X_test_scaled, y_test))

Trial 5 Complete [00h 00m 32s]
val_accuracy: 0.7965368032455444

Best val_accuracy So Far: 0.7965368032455444
Total elapsed time: 00h 05m 49s
INFO:tensorflow:Oracle triggered exit


In [ ]:
tuner.results_summary()

Results summary
Results in project/diabetes
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
num_layers: 9
units_0: 96
units_1: 288
learning_rate: 0.01
units_2: 32
units_3: 32
units_4: 32
units_5: 32
units_6: 32
units_7: 32
units_8: 32
Score: 0.7936507860819498
Trial summary
Hyperparameters:
num_layers: 17
units_0: 480
units_1: 192
learning_rate: 0.0001
units_2: 384
units_3: 160
units_4: 416
units_5: 192
units_6: 64
units_7: 384
units_8: 256
units_9: 32
units_10: 32
units_11: 32
units_12: 32
units_13: 32
units_14: 32
units_15: 32
units_16: 32
Score: 0.7922078172365824
Trial summary
Hyperparameters:
num_layers: 9
units_0: 480
units_1: 288
learning_rate: 0.0001
units_2: 448
units_3: 160
units_4: 352
units_5: 192
units_6: 192
units_7: 352
units_8: 320
units_9: 416
units_10: 96
units_11: 448
units_12: 352
units_13: 192
units_14: 512
units_15: 480
units_16: 96
Score: 0.7893218000729879
Trial summary
Hyperparameters:
num_layers: 19
units_0

In [18]:
# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units= 96, kernel_initializer='he_uniform',activation='relu',input_shape= (8,)))

# Adding the second hidden layer
classifier.add(Dense(units= 288, kernel_initializer='he_uniform',activation='relu'))
classifier.add(Dense(units= 32, kernel_initializer='he_uniform',activation='relu'))
classifier.add(Dense(units= 32, kernel_initializer='he_uniform',activation='relu'))
classifier.add(Dense(units= 32, kernel_initializer='he_uniform',activation='relu'))
classifier.add(Dense(units= 32, kernel_initializer='he_uniform',activation='relu'))
classifier.add(Dense(units= 32, kernel_initializer='he_uniform',activation='relu'))
classifier.add(Dense(units= 32, kernel_initializer='he_uniform',activation='relu'))
classifier.add(Dense(units= 32, kernel_initializer='he_uniform',activation='relu'))

# Adding the output layer
classifier.add(Dense(units= 1, kernel_initializer='glorot_uniform', activation = 'sigmoid'))

# Compiling the ANN
optimizer = tf.keras.optimizers.Adam()

classifier.compile(optimizer = Adam(learning_rate=0.01), loss = 'binary_crossentropy', metrics = ['accuracy'])

# Fitting the ANN to the Training set
model_history=classifier.fit(X_train, y_train,validation_split=0.33, batch_size = 10, epochs = 100)

Epoch 1/100
36/36 [==============================] - 1s 8ms/step - loss: 13.6043 - accuracy: 0.4568 - val_loss: 0.7809 - val_accuracy: 0.6180
Epoch 2/100
36/36 [==============================] - 0s 3ms/step - loss: 0.9182 - accuracy: 0.5682 - val_loss: 0.8174 - val_accuracy: 0.5674
Epoch 3/100
36/36 [==============================] - 0s 3ms/step - loss: 1.8557 - accuracy: 0.5850 - val_loss: 1.7072 - val_accuracy: 0.6629
Epoch 4/100
36/36 [==============================] - 0s 3ms/step - loss: 0.9490 - accuracy: 0.5738 - val_loss: 0.6637 - val_accuracy: 0.6742
Epoch 5/100
36/36 [==============================] - 0s 4ms/step - loss: 0.6847 - accuracy: 0.6490 - val_loss: 0.6215 - val_accuracy: 0.6854
Epoch 6/100
36/36 [==============================] - 0s 3ms/step - loss: 0.6386 - accuracy: 0.6518 - val_loss: 0.6475 - val_accuracy: 0.6742
Epoch 7/100
36/36 [==============================] - 0s 3ms/step - loss: 0.6138 - accuracy: 0.6657 - val_loss: 0.6314 - val_accuracy: 0.6910
Epoch 8/100


In [14]:
classifier.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 96)                864       
                                                                 
 dense_7 (Dense)             (None, 288)               27936     
                                                                 
 dense_8 (Dense)             (None, 32)                9248      
                                                                 
 dense_9 (Dense)             (None, 32)                1056      
                                                                 
 dense_10 (Dense)            (None, 32)                1056      
                                                                 
 dense_11 (Dense)            (None, 32)                1056      
                                                                 
 dense_12 (Dense)            (None, 32)               

In [19]:
y_pred = classifier.predict(X_test_scaled)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

# Calculate the Accuracy
from sklearn.metrics import accuracy_score
score=accuracy_score(y_pred,y_test)

print(cm)
print(score)

[[157   0]
 [ 74   0]]
0.6796536796536796
